In [1]:
skip_training = False  # Set this flag to True to skip training the model

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from transformers import BertTokenizer, BertModel, AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader

import pandas as pd

import numpy as np

import time
import random
import functools
import os

Material:  
https://www.vinai.io/phobert-the-first-public-large-scale-language-models-for-vietnamese

# 1. Read the data

https://github.com/datquocnguyen/VnDT#data-split

https://github.com/datquocnguyen/VnDT/blob/master/VnDT-paper-CameraReadyVersion.pdf

## 1.1 Read the tags

In [3]:
import re
def read_1(file_name):
    with open(file_name, encoding='utf8') as f:
        # line = ['Np Proper noun','Nc Classifier noun',...]
        lines = re.split('\n',f.read())
    
    tags = []
    tag_dict = {}
    for line in lines:
        tmp = line.split(' ',1)
        tags.append(tmp[0])
        tag_dict[tmp[0]] = tmp[1]
        
    return tags, tag_dict

## 1.2 Read train - dev - test corpuses

In [4]:
import re
def read_2(file_name):
    """
    Parameters
    ----------
    file_name - string
        a path to a file with an annotated corpus
    
    Returns 
    -------
    words - a list of lists of words
    tags - a list of lists of tags
        For example, the first sentence in a file is word1-tag1, word2-tag2 
        and the next sentence is word3_/_tag3. Then you should get:
        words = [['word1','word2'],['word3']]
        tags = [['tag1','tag2'],['tag3']]
    """
    
    df = pd.read_csv(file_name, sep='\t|\n', names = ['idx','word','c3','c4','tag','c6','c7','c8','c9','c10'])
    
    words = []
    tags = []
    sentence_words = []
    sentence_tags = []
    prev_idx = 0
    for _,row in df.iterrows():
        if row['idx'] != prev_idx+1:
            words.append(sentence_words)
            tags.append(sentence_tags)
            sentence_words = []
            sentence_tags = []
            
        sentence_words.append(row['word'])
        sentence_tags.append(row['tag'])
        prev_idx = row['idx']
            
    return words, tags

## 1.3 Load the data

In [5]:
cwd = os.getcwd()
cwd

'/m/home/home1/12/dangp1/unix/POS_NER'

In [6]:
tags_vocab_path = cwd+'/data/tags_vocab.txt'
train_path = cwd+'/data/train.txt'
valid_path = cwd+'/data/dev.txt'
test_path = cwd+'/data/test.txt'

tags_vocab, tags_vocab_dict = read_1(tags_vocab_path)
train_words, train_tags = read_2(train_path)
valid_words, valid_tags = read_2(valid_path)
test_words, test_tags = read_2(test_path)

print(len(train_words))
print(len(valid_words))
print(len(test_words))

/u/12/dangp1/unix/.conda/envs/concac/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


8976
199
1019


## 1.4 Enumerate labels

In [7]:
def tag2num(tags_vocab):
    return dict(zip( tags_vocab, range(1, len(tags_vocab)+1) ))

tag2num = tag2num(tags_vocab)

In [8]:
def Enumerate_tags(sentence_tags, tag2num):
    converted_tags = [ list(map(lambda x: tag2num[x], sentence_tag)) for sentence_tag in sentence_tags ]
    return converted_tags

enumerated_train_tags = Enumerate_tags(train_tags, tag2num)
enumerated_valid_tags = Enumerate_tags(valid_tags, tag2num)
enumerated_test_tags = Enumerate_tags(test_tags, tag2num)

In [9]:
PAD_IDX = 0

# 2. Study the data

# 3. Encode

In [10]:
class Encode(Dataset):
    def __init__(self, words, tags, tags_vocab, tokenizer, max_len, pad_idx):
        self.data = words
        self.labels = tags
        self.tags_vocab = tags_vocab
        self.tokenizer = tokenizer
        self.len = len(self.data)
        self.max_len = max_len
        self.pad_idx = pad_idx
        
    def __getitem__(self, index):
        inputs = self.tokenizer.encode_plus(
            self.data[index],
            None,
            add_special_tokens=False,
            max_length=self.max_len,
            padding = 'max_length',
            truncation=True
        )
        
        ids = inputs['input_ids']
        
        labels = self.labels[index] 
        labels += [self.pad_idx]*(self.max_len-len(labels))      # pad to the right
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'labels': torch.tensor(self.labels[index], dtype=torch.long)
        }
    
    def __len__(self):
        return self.len

In [11]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32

In [12]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
training_set = Encode(train_words, enumerated_train_tags, tags_vocab, tokenizer, MAX_LEN, PAD_IDX)
validating_set = Encode(valid_words, enumerated_valid_tags, tags_vocab, tokenizer, MAX_LEN, PAD_IDX)
testing_set = Encode(test_words, enumerated_test_tags, tags_vocab, tokenizer, MAX_LEN, PAD_IDX)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

valid_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
validating_loader = DataLoader(validating_set, **valid_params)
testing_loader = DataLoader(testing_set, **test_params)

# 4. Building the model

In [14]:
#https://github.com/VinAIResearch/PhoBERT
phobert = AutoModel.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [16]:
#https://github.com/bentrevett/pytorch-pos-tagging/blob/master/2_transformer.ipynb
class PhoBERTPoSTagger(nn.Module):
    def __init__(self,
                 bert,
                 output_dim, 
                 dropout):
        
        super(PhoBERTPoSTagger, self).__init__()
        self.bert = bert
        for params in self.bert.parameters():
            params.requires_grad =  True
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.fc1 = nn.Linear(embedding_dim, 512)
        self.fc2 = nn.Linear(512,256 )
        self.fc = nn.Linear(256, output_dim)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, ids):
  
        # ids = (batch_size, max_len)
    
        outputs = self.bert(
            input_ids=ids,
        )
        
        # outputs = BERT return
    
        outputs = outputs[0]
        
        # outputs = (batch_size, max_len, hidden_size)
        
        outputs = F.relu(self.fc1(self.dropout1(outputs)))
        outputs =  F.relu(self.fc2(self.dropout2(outputs)))
        outputs =  self.fc(self.dropout(outputs))
        
        # outputs = (batch_size, max_len, output_dim)
        
        outputs = outputs.permute(1,0,2)
        
        # outputs = (max_len, batch_size, output_dim)
        
        return outputs

In [17]:
output_dim = len(tags_vocab)+1
dropout = 0.25

model = PhoBERTPoSTagger(phobert,
                      output_dim, 
                      dropout)

# 5. Train the model

In [18]:
from torch.nn import CrossEntropyLoss

In [19]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [20]:
def compute_accuracy(preds, labels, pad_idx):
    max_pred_idx = preds.argmax(dim=1, keepdim = True)
    tags_idx = (labels != pad_idx).nonzero()
    correct = max_pred_idx[tags_idx].squeeze(1).eq(labels[tags_idx])
    return correct.sum() / torch.FloatTensor([labels[tags_idx].shape[0]]).to(device)

In [21]:
def train(model, training_loader, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
        
    model.train()
    
    for _,batch in enumerate(training_loader):
        optimizer.zero_grad()
        #model.zero_grad()
                
        input_ids = batch['ids'].to(device)
        labels = batch['labels'].to(device)
                
        outputs = model(input_ids)
                
        # outputs = (max_len, batch_size, output_dim)
        # labels = (batch_size, max_len)
                
        labels = labels.permute(1,0)
        # labels = (max_len, batch_size)
                
        outputs = outputs.contiguous().view(-1, outputs.shape[-1])
                
        labels = labels.contiguous().view(-1)
                
        # outputs = (max_len*batch_size, output_dim)
        # labels = (max_len*batch_size)
                
        # Note: Seperating words into sentences is not necessary anymore, 
        #       we only care if an output word matches its label
                                
        loss = criterion(outputs, labels)
        acc = compute_accuracy(outputs, labels, tag_pad_idx)
                
        loss.backward()
        optimizer.step()
                
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(training_loader), epoch_acc / len(training_loader)


In [22]:
def evaluate(model, validating_loader, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
        
    model.eval()
    
    with torch.no_grad():
        for _,batch in enumerate(validating_loader):
            input_ids = batch['ids'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids)

            # outputs = (max_len, batch_size, output_dim)
            # labels = (batch_size, max_len)

            labels = labels.permute(1,0)
            # labels = (max_len, batch_size)

            outputs = outputs.contiguous().view(-1, outputs.shape[-1])

            labels = labels.contiguous().view(-1)

            # outputs = (max_len*batch_size, output_dim)
            # labels = (max_len*batch_size)

            # Note: Seperating words into sentences is not necessary anymore, 
            #       we only care if an output word matches its label

            loss = criterion(outputs, labels)
            acc = compute_accuracy(outputs, labels, 0)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(validating_loader), epoch_acc / len(validating_loader)


In [23]:
criterion = CrossEntropyLoss(ignore_index = PAD_IDX)
N_EPOCHS = 40
LEARNING_RATE = 1e-05
optimizer = optim.AdamW(model.parameters(), lr = LEARNING_RATE)

model = model.to(device)
criterion = criterion.to(device)

In [24]:
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch+1, N_EPOCHS))
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, training_loader, optimizer, criterion, PAD_IDX)
    valid_loss, valid_acc = evaluate(model, validating_loader, criterion, PAD_IDX)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'postag-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


======== Epoch 1 / 40 ========
Epoch: 01 | Epoch Time: 2m 20s
	Train Loss: 2.381 | Train Acc: 37.86%
	 Val. Loss: 1.450 |  Val. Acc: 67.26%

======== Epoch 2 / 40 ========
Epoch: 02 | Epoch Time: 2m 22s
	Train Loss: 1.048 | Train Acc: 75.76%
	 Val. Loss: 0.625 |  Val. Acc: 85.42%

======== Epoch 3 / 40 ========
Epoch: 03 | Epoch Time: 2m 22s
	Train Loss: 0.553 | Train Acc: 87.25%
	 Val. Loss: 0.395 |  Val. Acc: 90.71%

======== Epoch 4 / 40 ========
Epoch: 04 | Epoch Time: 2m 22s
	Train Loss: 0.400 | Train Acc: 90.18%
	 Val. Loss: 0.325 |  Val. Acc: 91.34%

======== Epoch 5 / 40 ========
Epoch: 05 | Epoch Time: 2m 22s
	Train Loss: 0.331 | Train Acc: 91.44%
	 Val. Loss: 0.278 |  Val. Acc: 92.55%

======== Epoch 6 / 40 ========
Epoch: 06 | Epoch Time: 2m 22s
	Train Loss: 0.284 | Train Acc: 92.42%
	 Val. Loss: 0.274 |  Val. Acc: 92.15%

======== Epoch 7 / 40 ========
Epoch: 07 | Epoch Time: 2m 22s
	Train Loss: 0.251 | Train Acc: 93.03%
	 Val. Loss: 0.244 |  Val. Acc: 92.84%

======== Epo

In [25]:
model.load_state_dict(torch.load('postag-model.pt'))

test_loss, test_acc = evaluate(model, testing_loader, criterion, PAD_IDX)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.206 | Test Acc: 94.89%
